# ACOS-ABSA: Unified Preprocessing Pipeline for ModernBERT

1. Il paper originale utilizza BERT-base-uncased (Devlin et al., 2018) come encoder principale. Sebbene rivoluzionario all'epoca, BERT ha un contesto limitato (512 token) e una capacità di generalizzazione inferiore rispetto ai modelli attuali.

    * **La Nostra Soluzione:** Utilizziamo ModernBERT-base, un modello State-of-the-Art (2024) addestrato su un corpus molto più vasto e con una finestra di contesto estesa (8k token).

    * **Vantaggio:** ModernBERT offre rappresentazioni contestuali più ricche, fondamentali per risolvere il problema degli Aspetti Impliciti (es. dedurre Price da "it is expensive"), che rappresentano una sfida critica nel dataset ACOS.
+1

2. Strategia di Allineamento Sub-word (Robust Tokenization)
Il paper gestisce l'allineamento tra parole e token, ma spesso i modelli standard soffrono quando una parola annotata (es. "difficulty") viene spezzata in più sub-token (diffic, ##ulty).

    * **La Nostra Soluzione:** Abbiamo implementato una pipeline di preprocessing personalizzata che utilizza i word_ids per mappare precisamente le etichette BIO sui sub-token.

    * **Vantaggio:** Questo garantisce che nessun'informazione venga persa durante la tokenizzazione: se una parola è un'Opinione, tutti i suoi frammenti (token) erediteranno correttamente l'etichetta, migliorando la Recall del modello.

3. Semplificazione Architetturale (Rimozione del CRF)
Il modello Extract-Classify-ACOS impiega un layer CRF (Conditional Random Field) sopra BERT per "pulire" la sequenza di tag predetti e imporre vincoli logici.

   * **La Nostra Soluzione:** Sfruttando la maggiore potenza di estrazione delle feature di ModernBERT, iniziamo con una Linear Classification Head standard.

   * **Vantaggio:** Questo riduce drasticamente la complessità computazionale e i tempi di addestramento/inferenza. La capacità superiore di ModernBERT di apprendere le dipendenze locali rende spesso superfluo l'uso di un CRF, permettendo al modello di apprendere i vincoli BIO direttamente dai dati.

4. Gestione "Native" degli Span Impliciti
Come evidenziato nel paper, una larga percentuale di quadruple contiene aspetti o opinioni implicite (Span Nulli).

    * **La Nostra Soluzione:** Il nostro preprocessing gestisce esplicitamente gli span (-1, -1) nel dataset, preparando il terreno per la fase successiva (Classificazione). Mentre la fase di estrazione corrente si concentra sugli span espliciti, i vettori [CLS] di ModernBERT sono già ottimizzati per catturare il contesto globale necessario a predire le categorie implicite nel secondo step della pipeline.

In [1]:
# Import delle librerie necessarie
import torch
import numpy as np
import random
import pandas as pd
import wandb
import os
import sys
import re
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer
import pickle
print("Librerie caricate.")

Librerie caricate.


### Impostazioni per la riproducibilità 

In [2]:
def set_seed(seed_value=42):
    """Imposto i seed per la riproducibilità."""
    random.seed(seed_value)
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)
    if torch.cuda.is_available():
        # Imposto anche i seed per la GPU, se disponibile
        torch.cuda.manual_seed_all(seed_value)

# Esegui l'impostazione del seed
set_seed(42) 
print("Random seeds impostati su 42.")

Random seeds impostati su 42.


## Raw Data Parsing & Quadruple Decoding

### Caricamento dei 3 dataset LAPTOP-ACOS e dei 3 dataset RESTAURANT-ACOS
Questa cella è dedicata al caricamento dei dataset TSV di ACOS (Training, Development e Test) dalla directory locale.

Data la struttura complessa del file di annotazione, che contiene tabulazioni (\t) interne e un numero variabile di quadruple per riga, la funzione load_as_single_string adotta una strategia di caricamento flessibile:

1. **Forzatura Stringa Unica:** Viene utilizzato un separatore inesistente (\x07) per istruire Pandas a caricare l'intera riga TSV come una singola colonna stringa *(full_line_data).*

2. **Robustezza:** Questo approccio previene i comuni ParserError causati da tabulazioni o delimitatori sporchi interni, garantendo che i dati grezzi vengano letti completamente senza perdita.

L'output di questa cella sono i tre DataFrame *(df_train, df_dev, df_test)*, ciascuno pronto per il parsing sequenziale sul contenuto della colonna full_line_data.

In [3]:

# Definisci il percorso per ACOS
DATA_DIR_ACOS = 'data/Laptop-ACOS/'

# Definisci i percorsi completi dei file
TRAIN_FILE_PATH_ACOS = os.path.join(DATA_DIR_ACOS, 'laptop_train_quad_bert.tsv')
DEV_FILE_PATH_ACOS = os.path.join(DATA_DIR_ACOS, 'laptop_dev_quad_bert.tsv')
TEST_FILE_PATH_ACOS = os.path.join(DATA_DIR_ACOS, 'laptop_test_quad_bert.tsv')

# Definisci il percorso REESTAURANT
DATA_DIR_RESTAURANT = 'data/Restaurant-ACOS/'

# Definisci i percorsi completi dei file
TRAIN_FILE_PATH_RESTAURANT = os.path.join(DATA_DIR_RESTAURANT, 'rest16_train_quad_bert.tsv')
DEV_FILE_PATH_RESTAURANT = os.path.join(DATA_DIR_RESTAURANT, 'rest16_dev_quad_bert.tsv')
TEST_FILE_PATH_RESTAURANT = os.path.join(DATA_DIR_RESTAURANT, 'rest16_test_quad_bert.tsv')

def load_as_single_string(path):
    """
    Carica l'intero file TSV in un'unica colonna stringa, forzando la lettura riga per riga, 
    per evitare errori di parsing dovuti a delimitatori interni.
    """
    # 1. Utilizza read_csv ma forza l'uso di un separatore inesistente e non usare header
    df = pd.read_csv(
        path, 
        sep='\x07', # Separa per un carattere inesistente (Bell character)
        header=None, 
        on_bad_lines='skip', # Ignora le righe che danno problemi di formattazione
        engine='python'
    )
    
    # Rinomina l'unica colonna che contiene l'intera riga TSV
    df.columns = ['full_line_data']    
    print(f"File {path.split('/')[-1]} caricato come {df.shape[1]} colonna unica.")
    return df

# Esecuzione del caricamento corretto
try:
    df_train_laptop = load_as_single_string(TRAIN_FILE_PATH_ACOS)
    df_dev_laptop = load_as_single_string(DEV_FILE_PATH_ACOS)
    df_test_laptop = load_as_single_string(TEST_FILE_PATH_ACOS)
    
    df_train_rest = load_as_single_string(TRAIN_FILE_PATH_RESTAURANT)
    df_dev_rest = load_as_single_string(DEV_FILE_PATH_RESTAURANT)
    df_test_rest = load_as_single_string(TEST_FILE_PATH_RESTAURANT)
    
    print("Caricamento Flessibile Completato.")
    
except Exception as e:
    print(f"Errore critico durante il caricamento: {e}")
    raise

print(df_train_laptop.shape, df_dev_laptop.shape, df_test_laptop.shape)
print(df_train_rest.shape, df_dev_rest.shape, df_test_rest.shape)

File laptop_train_quad_bert.tsv caricato come 1 colonna unica.
File laptop_dev_quad_bert.tsv caricato come 1 colonna unica.
File laptop_test_quad_bert.tsv caricato come 1 colonna unica.
File rest16_train_quad_bert.tsv caricato come 1 colonna unica.
File rest16_dev_quad_bert.tsv caricato come 1 colonna unica.
File rest16_test_quad_bert.tsv caricato come 1 colonna unica.
Caricamento Flessibile Completato.
(2934, 1) (326, 1) (816, 1)
(1530, 1) (171, 1) (583, 1)


### Pipeline di Parsing e Strutturazione dei Dati(Quadruple)
Questa sezione implementa la pipeline di pre-elaborazione finale che trasforma la riga grezza di input in un formato Python strutturato, pronto per la Tokenizzazione del modello ModernBERT.

Il codice esegue un processo a due fasi per ogni riga del dataset:

1. **Separazione (Parsing della Riga Grezza)**: Esegue la divisione della riga unica *(full_line_data)* utilizzando il separatore Tab (\t). Questo isola la recensione pulita *(review_text)* dalla stringa contenente le quadruple codificate (il target).

2. **Decodifica Strutturale (Deep Parsing)**: Applica la funzione *parse_target_quadruples* alla stringa target. Questa funzione scompone la stringa in un dizionario con chiavi chiare **(span_A, span_B, category_aspect, sentiment)**, gestendo gli Indici Span Nullo (-1,-1) e validando la struttura a 4 elementi.

L'output finale è il DataFrame pulito con la colonna parsed_quadruples, che contiene le informazioni di target necessarie per creare le label di Sequence Labeling nella fase successiva.

In [4]:
# --- I. DEFINIZIONE DELLE FUNZIONI DI PARSING ---

def parse_target_quadruples(target_string):
    """
    Decodifica la stringa di una singola quadrupla in un dizionario strutturato.
    """
    if not target_string:
        return {} 

    parts = target_string.split()
    if len(parts) != 4:
        return {}
    
    span_A_str, category_aspect, sentiment_str, span_B_str = parts

    def parse_span(span_str):
        if span_str == '-1,-1':
            return (-1, -1)
        try:
            start, end = map(int, span_str.split(','))
            return (start, end)
        except ValueError:
            return (-1, -1)

    span_A = parse_span(span_A_str)
    span_B = parse_span(span_B_str)
    
    try:
        sentiment = int(sentiment_str)
    except ValueError:
        sentiment = -1 

    return {
        'span_A': span_A,
        'span_B': span_B,
        'category_aspect': category_aspect,
        'sentiment': sentiment
    }

def apply_deep_parsing(raw_quadruples_list):
    return [parse_target_quadruples(q) for q in raw_quadruples_list]

def parse_full_line(full_line_string):
    parts = full_line_string.split('\t', 1) 
    if len(parts) != 2:
        return full_line_string.strip(), []

    review_text = parts[0].strip()
    target_string_raw = parts[1].strip()
    
    # Pulizia residui numerici a fine riga
    target_string_clean = re.sub(r'[\r\n\s]\d+$', '', target_string_raw).strip()
    raw_quadruples_list = [q.strip() for q in target_string_clean.split('\t') if q.strip()]

    return review_text, raw_quadruples_list

def apply_full_parsing_pipeline(df):
    """
    Applica la pipeline di parsing restituendo un nuovo DataFrame processato.
    """
    df_res = df.copy() 
    
    # Passaggio 1: Split testo e liste di stringhe target
    parsed_lines = df_res['full_line_data'].apply(parse_full_line)
    df_res['review_text'] = parsed_lines.apply(lambda x: x[0])
    df_res['raw_quadruples_list'] = parsed_lines.apply(lambda x: x[1])
    
    # Passaggio 2: Decodifica delle quadruple in dizionari
    df_res['parsed_quadruples'] = df_res['raw_quadruples_list'].apply(apply_deep_parsing)
    
    # Pulizia colonne temporanee e originali
    return df_res.drop(columns=['full_line_data', 'raw_quadruples_list'])


# --- II. ESECUZIONE CON COPIA DEI DATASET ---

print("Creazione delle copie e avvio del parsing...")

# 1. Creazione delle copie dedicate al parsing
df_train_parsing_laptop = df_train_laptop.copy()
df_dev_parsing_laptop = df_dev_laptop.copy()
df_test_parsing_laptop = df_test_laptop.copy()

df_train_parsing_rest = df_train_rest.copy()
df_dev_parsing_rest = df_dev_rest.copy()
df_test_parsing_rest = df_test_rest.copy()

# 2. Applicazione della pipeline sulle nuove variabili
df_train_parsing_laptop = apply_full_parsing_pipeline(df_train_parsing_laptop)
df_dev_parsing_laptop = apply_full_parsing_pipeline(df_dev_parsing_laptop)
df_test_parsing_laptop = apply_full_parsing_pipeline(df_test_parsing_laptop)

df_train_parsing_rest = apply_full_parsing_pipeline(df_train_parsing_rest)
df_dev_parsing_rest = apply_full_parsing_pipeline(df_dev_parsing_rest)
df_test_parsing_rest = apply_full_parsing_pipeline(df_test_parsing_rest)

print("Parsing completato con successo.")

# Definiamo il nome della cartella
cartella_parsing = "data_parsing"

# Creiamo la cartella se non esiste
os.makedirs(cartella_parsing, exist_ok=True)

print(f"Salvataggio dei dataset parsati nella cartella '{cartella_parsing}'...")

# --- SALVATAGGIO LAPTOP ---
df_train_parsing_laptop.to_pickle(os.path.join(cartella_parsing, "train_laptop_parsed.pkl"))
df_dev_parsing_laptop.to_pickle(os.path.join(cartella_parsing, "dev_laptop_parsed.pkl"))
df_test_parsing_laptop.to_pickle(os.path.join(cartella_parsing, "test_laptop_parsed.pkl"))
print("Dataset Laptop salvati!")

# --- SALVATAGGIO RESTAURANT ---
df_train_parsing_rest.to_pickle(os.path.join(cartella_parsing, "train_rest_parsed.pkl"))
df_dev_parsing_rest.to_pickle(os.path.join(cartella_parsing, "dev_rest_parsed.pkl"))
df_test_parsing_rest.to_pickle(os.path.join(cartella_parsing, "test_rest_parsed.pkl"))
print("Dataset Restaurant salvati!")




# --- III. VERIFICA FINALE ---

print("\nAnteprima del DataFrame di Test (df_test_parsing_laptop):")
display(df_test_parsing_laptop[['review_text', 'parsed_quadruples']].head())

print("\nAnteprima del DataFrame di Test (df_test_parsing_rest):")
display(df_test_parsing_rest[['review_text', 'parsed_quadruples']].head())

# Controllo specifico richiesto (riga 3)
print(f"\nQuadruple decodificate: {df_test_parsing_rest['parsed_quadruples'].loc[3]}")
print(f"Testo corrispondente: {df_test_parsing_rest['review_text'].loc[3]}")

print("\nDimensioni finali (Train, Dev, Test):")
print(df_train_parsing_laptop.shape, df_dev_parsing_laptop.shape, df_test_parsing_laptop.shape)
print(df_train_parsing_rest.shape, df_dev_parsing_rest.shape, df_test_parsing_rest.shape)

Creazione delle copie e avvio del parsing...
Parsing completato con successo.
Salvataggio dei dataset parsati nella cartella 'data_parsing'...
Dataset Laptop salvati!
Dataset Restaurant salvati!

Anteprima del DataFrame di Test (df_test_parsing_laptop):


,review_text,parsed_quadruples
0,"the unit cost $ 275 to start with , so it is n...","[{'span_A': (1, 2), 'span_B': (12, 14), 'categ..."
1,going from ace ##r 15 to ace ##r 11 was diffic...,"[{'span_A': (6, 9), 'span_B': (10, 11), 'categ..."
2,also it ' s not a true ss ##d drive in there b...,"[{'span_A': (7, 10), 'span_B': (-1, -1), 'cate..."
3,the computer has difficulty switching between ...,"[{'span_A': (1, 2), 'span_B': (3, 4), 'categor..."
4,2 / 28 / 18 - a couple days ago i updated the ...,"[{'span_A': (13, 15), 'span_B': (-1, -1), 'cat..."



Anteprima del DataFrame di Test (df_test_parsing_rest):


,review_text,parsed_quadruples
0,yu ##m !,"[{'span_A': (-1, -1), 'span_B': (0, 2), 'categ..."
1,serves really good su ##shi .,"[{'span_A': (3, 5), 'span_B': (2, 3), 'categor..."
2,not the biggest portions but adequate .,"[{'span_A': (3, 4), 'span_B': (0, 3), 'categor..."
3,green tea cr ##eme br ##ule ##e is a must !,"[{'span_A': (0, 7), 'span_B': (9, 10), 'catego..."
4,it has great su ##shi and even better service .,"[{'span_A': (3, 5), 'span_B': (2, 3), 'categor..."



Quadruple decodificate: [{'span_A': (0, 7), 'span_B': (9, 10), 'category_aspect': 'FOOD#QUALITY', 'sentiment': 2}]
Testo corrispondente: green tea cr ##eme br ##ule ##e is a must !

Dimensioni finali (Train, Dev, Test):
(2934, 2) (326, 2) (816, 2)
(1530, 2) (171, 2) (583, 2)


## Sub-word Alignment & BIO Labeling

### Pre-processing dei Dati: Tokenizzazione, Allineamento BIO e Identificazione degli Impliciti

Questa cella esegue la preparazione fondamentale del dataset testuale per renderlo compatibile con l'architettura Multi-Task basata su ModernBERT. Il codice processa le quadruple originali e trasforma le parole in tensori numerici, risolvendo il problema dell'allineamento dei token e implementando la logica di estrazione avanzata del task ACOS. 

Nello specifico, la funzione `encode_and_align_labels` esegue le seguenti operazioni chiave:

* **Tokenizzazione e Allineamento Sub-word:** Poiché i modelli BERT suddividono le parole in sotto-unità (sub-words), il codice riallinea le etichette originali ai nuovi token generati. Utilizza lo schema di tagging BIO (Begin-Inside-Outside) per mappare accuratamente le parole esplicite, assegnando `B-ASP`/`I-ASP` per gli aspetti e `B-OPI`/`I-OPI` per le opinioni.
* **Gestione degli Elementi Impliciti (Il Core del Task ACOS):** Le recensioni dei prodotti contengono una grande quantità di aspetti e opinioni implicite. Per gestire questi casi, se un elemento non è esplicitamente espresso nel testo, viene descritto con il valore NULL (rappresentato dalle coordinate `-1, -1` nel nostro dataset).
* **Creazione delle Etichette Binarie:** Quando il sistema rileva coordinate `(-1, -1)`, accende due flag binari (`has_implicit_aspect` e `has_implicit_opinion`). Queste etichette extra verranno fornite al modello in fase di addestramento per addestrare i due classificatori binari posizionati sul token `[CLS]`, permettendo alla rete di prevedere la presenza di aspetti o opinioni implicite.
* **Salvataggio Seriale:** Infine, i dati processati e allineati per entrambi i domini (Laptop e Restaurant) vengono salvati in formato `.pkl` all'interno della cartella `data_allineati`, pronti per essere caricati in modo efficiente dai DataLoader.

In [5]:
# 1. Caricamento del Tokenizer di ModernBERT
tokenizer_name = "answerdotai/ModernBERT-base"
tokenizer = AutoTokenizer.from_pretrained(tokenizer_name)

def encode_and_align_labels(words, parsed_quadruples, tokenizer, max_len=128):
    tokenized_input = tokenizer(
        words, truncation=True, max_length=max_len,
        padding='max_length', is_split_into_words=True
    )

    labels = ["O"] * max_len
    word_ids = tokenized_input.word_ids()
    
    # --- GESTIONE IMPLICITI ---
    has_implicit_aspect = 0
    has_implicit_opinion = 0
    
    for quad in parsed_quadruples:
        asp_span = quad.get('span_A', (-1, -1))
        opi_span = quad.get('span_B', (-1, -1))
        
        # Se troviamo le coordinate (-1, -1), accendiamo la spia dell'implicito!
        if asp_span == (-1, -1) or asp_span == [-1, -1]:
            has_implicit_aspect = 1
        if opi_span == (-1, -1) or opi_span == [-1, -1]:
            has_implicit_opinion = 1
            
        def align_span(span, b_tag, i_tag):
            if span == (-1, -1) or span == [-1, -1]:
                return
            
            start_word_idx, end_word_idx = span
            span_started = False
            
            for i, word_id in enumerate(word_ids):
                if i >= max_len or word_id is None:
                    continue
                if start_word_idx <= word_id < end_word_idx:
                    if not span_started:
                        labels[i] = b_tag
                        span_started = True
                    else:
                        labels[i] = i_tag

        align_span(asp_span, "B-ASP", "I-ASP")
        align_span(opi_span, "B-OPI", "I-OPI")

    label_map = {"O": 0, "B-ASP": 1, "I-ASP": 2, "B-OPI": 3, "I-OPI": 4}
    label_ids = [label_map[l] for l in labels]

    return {
        'input_ids': tokenized_input['input_ids'],
        'attention_mask': tokenized_input['attention_mask'],
        'labels': label_ids,
        'implicit_aspect_label': has_implicit_aspect, 
        'implicit_opinion_label': has_implicit_opinion 
    }
    


def process_align_dataset(df, tokenizer, split_name):
    """
    Applica l'allineamento e aggiunge le colonne input_ids, attention_mask e labels.
    """
    print(f"Allineamento in corso per lo split: {split_name}...")
    
    # Applichiamo la funzione riga per riga
    # Usiamo .split() perché il testo è pre-tokenizzato nel dataset originale
    processed_series = df.apply(
        lambda row: encode_and_align_labels(
            row['review_text'].split(), 
            row['parsed_quadruples'], 
            tokenizer, 
            max_len=128
        ), 
        axis=1
    )
    
    # Convertiamo la lista di dizionari in un DataFrame e lo concateniamo
    df_result = pd.DataFrame(processed_series.tolist(), index=df.index)
    return pd.concat([df, df_result], axis=1)



# --- 1. CREAZIONE DELLE COPIE DEI DATASET ---
# Creiamo copie profonde per evitare di modificare i DataFrame originali
df_train_align_laptop = df_train_parsing_laptop.copy()
df_dev_align_laptop = df_dev_parsing_laptop.copy()
df_test_align_laptop = df_test_parsing_laptop.copy()

df_train_align_rest = df_train_parsing_rest.copy()
df_dev_align_rest = df_dev_parsing_rest.copy()
df_test_align_rest = df_test_parsing_rest.copy()

# --- 2. ESECUZIONE DELL'ALLINEAMENTO ---
df_train_align_laptop = process_align_dataset(df_train_align_laptop, tokenizer, "TRAIN_LAPTOP")
df_dev_align_laptop = process_align_dataset(df_dev_align_laptop, tokenizer, "DEV_LAPTOP")
df_test_align_laptop = process_align_dataset(df_test_align_laptop, tokenizer, "TEST_LAPTOP")

df_train_align_rest = process_align_dataset(df_train_align_rest, tokenizer, "TRAIN_RESTAURANT")
df_dev_align_rest = process_align_dataset(df_dev_align_rest, tokenizer, "DEV_RESTAURANT")
df_test_align_rest = process_align_dataset(df_test_align_rest, tokenizer, "TEST_RESTAURANT")

# --- 3. OUTPUT DELLE PRIME RIGHE ---
print("\n" + "="*50)
print("VISUALIZZAZIONE DEI DATASET ALLINEATI")

print("\n--- PRIME RIGHE TRAIN LAPTOP---")
display(df_train_align_laptop[['review_text', 'input_ids', 'labels']].head())

print("\n--- PRIME RIGHE DEV LAPTOP ---")
display(df_dev_align_laptop[['review_text', 'input_ids', 'labels']].head())

print("\n--- PRIME RIGHE TEST LAPTOP ---")
display(df_test_align_laptop[['review_text', 'input_ids', 'labels']].head())


print("\n--- PRIME RIGHE TRAIN RESTAURANT---")
display(df_train_align_rest[['review_text', 'input_ids', 'labels']].head())

print("\n--- PRIME RIGHE DEV RESTAURANT ---")
display(df_dev_align_rest[['review_text', 'input_ids', 'labels']].head())

print("\n--- PRIME RIGHE TEST RESTAURANT ---")
display(df_test_align_rest[['review_text', 'input_ids', 'labels']].head())

# --- 4. SALVATAGGIO DEI DATASET NELLA CARTELLA "data_allineati" ---
# Creiamo la cartella se non esiste già
output_dir = "data_allineati"
os.makedirs(output_dir, exist_ok=True)

print(f"\nSalvataggio dei dataset nella cartella '{output_dir}' in formato Pickle...")

# Salviamo i dataset dei Laptop
df_train_align_laptop.to_pickle(os.path.join(output_dir, "train_laptop_aligned.pkl"))
df_dev_align_laptop.to_pickle(os.path.join(output_dir, "dev_laptop_aligned.pkl"))
df_test_align_laptop.to_pickle(os.path.join(output_dir, "test_laptop_aligned.pkl"))

# Salviamo i dataset dei Restaurant
df_train_align_rest.to_pickle(os.path.join(output_dir, "train_rest_aligned.pkl"))
df_dev_align_rest.to_pickle(os.path.join(output_dir, "dev_rest_aligned.pkl"))
df_test_align_rest.to_pickle(os.path.join(output_dir, "test_rest_aligned.pkl"))

print("✅ Salvataggio completato con successo! Dati pronti per il training.")


Allineamento in corso per lo split: TRAIN_LAPTOP...
Allineamento in corso per lo split: DEV_LAPTOP...
Allineamento in corso per lo split: TEST_LAPTOP...
Allineamento in corso per lo split: TRAIN_RESTAURANT...
Allineamento in corso per lo split: DEV_RESTAURANT...
Allineamento in corso per lo split: TEST_RESTAURANT...

VISUALIZZAZIONE DEI DATASET ALLINEATI

--- PRIME RIGHE TRAIN LAPTOP---


,review_text,input_ids,labels
0,ace ##r wants $ 170 to just look at it then ad...,"[50281, 584, 817, 83, 88, 1103, 5, 15046, 936,...","[0, 1, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,update : i repaired it myself for $ 12 .,"[50281, 11183, 27, 74, 4762, 12260, 262, 17089...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,i had nothing to lose since it was a paper wei...,"[50281, 74, 10178, 26142, 936, 77, 583, 17480,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,the shame of it is knowing it took me 15 minut...,"[50281, 783, 1200, 482, 1171, 262, 261, 14428,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,first one that they shipped was obviously defe...,"[50281, 7053, 531, 3529, 9328, 1200, 6390, 423...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 4, 4, 0, 0, ..."



--- PRIME RIGHE DEV LAPTOP ---


,review_text,input_ids,labels
0,this unit is ` ` pretty ` ` and st ##yl ##ish ...,"[50281, 2520, 8522, 261, 65, 65, 38256, 65, 65...","[0, 0, 1, 0, 0, 0, 3, 0, 0, 0, 3, 4, 4, 4, 4, ..."
1,for now i ' m okay with up ##ping the experien...,"[50281, 1542, 2666, 74, 8, 78, 536, 333, 3113,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,"seems unlikely but whatever , i ' ll go with it .","[50281, 339, 3030, 328, 10355, 2858, 38499, 13...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,this version has been my least favorite versio...,"[50281, 2520, 4149, 7110, 20394, 2577, 38462, ...","[0, 0, 0, 0, 0, 0, 3, 4, 4, 1, 0, 0, 0, 0, 0, ..."
4,- biggest disappointment is the track pad .,"[50281, 14, 2760, 3219, 3431, 9626, 420, 261, ...","[0, 0, 0, 0, 3, 4, 4, 0, 0, 1, 2, 0, 0, 0, 0, ..."



--- PRIME RIGHE TEST LAPTOP ---


...


--- PRIME RIGHE DEV RESTAURANT ---


,review_text,input_ids,labels
0,ca n ' t wait wait for my next visit .,"[50281, 6357, 79, 8, 85, 14061, 14061, 1542, 2...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,"their sake list was extensive , but we were lo...","[50281, 14094, 84, 640, 3550, 4238, 2068, 3134...","[0, 0, 1, 2, 2, 0, 3, 4, 0, 0, 0, 0, 0, 0, 0, ..."
2,the spicy tuna roll was unusually good and the...,"[50281, 783, 1033, 2576, 85, 9821, 1811, 4238,...","[0, 0, 1, 2, 2, 2, 2, 0, 0, 0, 3, 0, 0, 1, 2, ..."
3,we love th pink pony .,"[50281, 664, 26617, 394, 49723, 81, 2421, 15, ...","[0, 0, 3, 0, 1, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,this place has got to be the best japanese res...,"[50281, 2520, 5070, 7110, 19559, 936, 1257, 78...","[0, 0, 1, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, ..."



--- PRIME RIGHE TEST RESTAURANT ---


,review_text,input_ids,labels
0,yu ##m !,"[50281, 30838, 817, 78, 2, 50282, 50283, 50283...","[0, 3, 4, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,serves really good su ##shi .,"[50281, 1498, 265, 28235, 12311, 3467, 817, 41...","[0, 0, 0, 0, 3, 1, 2, 2, 0, 0, 0, 0, 0, 0, 0, ..."
2,not the biggest portions but adequate .,"[50281, 1439, 783, 2760, 3219, 631, 621, 2858,...","[0, 3, 4, 4, 4, 1, 2, 0, 3, 4, 0, 0, 0, 0, 0, ..."
3,green tea cr ##eme br ##ule ##e is a must !,"[50281, 11707, 442, 66, 7083, 817, 20867, 1288...","[0, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 0, 3, ..."
4,it has great su ##shi and even better service .,"[50281, 262, 7110, 17124, 3467, 817, 41386, 39...","[0, 0, 0, 3, 1, 2, 2, 0, 0, 3, 1, 0, 0, 0, 0, ..."



Salvataggio dei dataset nella cartella 'data_allineati' in formato Pickle...
✅ Salvataggio completato con successo! Dati pronti per il training.


### Data Preparation & Offset Fix ( Category & Sentiment)

Per replicare l'approccio *Multiple Multi-class Classification* del paper, il codice è diviso in tre funzioni modulari:

1. **Creazione del Vocabolario (`build_category_vocab`)**: 
   Scansiona tutti i set di dati (Train, Dev, Test) per estrarre l'elenco completo delle categorie uniche del dominio. Le ordina alfabeticamente per garantire che l'indice assegnato a ciascuna categoria (es. `DISPLAY#QUALITY` -> Indice 24) sia identico e immutabile in tutta la pipeline.

2. **Prodotto Cartesiano e Labels (`prepare_pairs_dataset`)**:
   * **Prodotto Cartesiano**: Per ogni frase, crea tutte le combinazioni possibili tra gli aspetti e le opinioni presenti.
   * **Fix Offset `[CLS]`**: Applica automaticamente un correttivo matematico (`+1`) agli indici salvati per allinearli al token speciale `[CLS]` che ModernBERT aggiungerà in fase di addestramento. Questo garantisce che gli span puntino sempre alle parole corrette.
   * **Vettorizzazione Multi-Classe**: Per ogni coppia creata, genera un array lungo quanto il numero di categorie (es. 121 per Laptop), inizializzato interamente a `3` (Classe *Invalid*). Se la coppia corrisponde a una quadrupla d'oro, sovrascrive il `3` con il sentiment corretto (`0=Pos`, `1=Neg`, `2=Neu`) all'indice della categoria corrispondente.

3. **`process_and_save_domain`**:
   Una funzione che automatizza l'intera pipeline per un dominio specifico (Laptop o Restaurant) e salva i nuovi dataset strutturati (e la lista delle categorie) all'interno della cartella `data_coppie` in formato `.pkl`.

In [6]:
import pandas as pd
import os
import pickle

# --- 1. FUNZIONE PER CREARE IL VOCABOLARIO DELLE CATEGORIE ---
def build_category_vocab(df_list):
    """
    Estrae tutte le categorie uniche da una lista di dataframe e le ordina.
    Ritorna la lista delle categorie e il dizionario {categoria: id}.
    """
    all_categories = set()
    for df in df_list:
        for quads in df['parsed_quadruples']:
            for q in quads:
                all_categories.add(q['category_aspect'])
                
    category_list = sorted(list(all_categories))
    category2id = {c: idx for idx, c in enumerate(category_list)}
    
    return category_list, category2id


# --- 2. FUNZIONE PER IL PRODOTTO CARTESIANO E LE LABELS (AGGIORNATA PER SOTA) ---
def prepare_pairs_dataset(df, category_list, category2id, invalid_class=3):
    """
    Crea le coppie candidate per ogni frase.
    NOVITÀ: Introduce l'Hard Negative Sampling per gli impliciti.
    """
    new_data = []
    
    for idx, row in df.iterrows():
        text = row['review_text']
        quads = row['parsed_quadruples']
        
        aspect_spans = set()
        opinion_spans = set()
        
        # 🔥 IL SEGRETO PER SUPERARE IL PAPER: HARD NEGATIVE SAMPLING 🔥
        # Aggiungiamo forzatamente le coordinate implicite a TUTTE le frasi.
        # Così il modello impara a dire "NO" (Classe 3) quando incrocia un
        # implicito dove non dovrebbe esserci!
        aspect_spans.add((-1, -1))
        opinion_spans.add((-1, -1))
        
        for q in quads:
            aspect_spans.add(tuple(q.get('span_A', (-1, -1))))
            opinion_spans.add(tuple(q.get('span_B', (-1, -1))))
            
        for a_span in aspect_spans:
            for o_span in opinion_spans:
                
                # Fix Offset +1 per il token [CLS]
                fixed_a = (a_span[0] + 1 if a_span[0] != -1 else -1, 
                           a_span[1] + 1 if a_span[1] != -1 else -1)
                
                fixed_o = (o_span[0] + 1 if o_span[0] != -1 else -1, 
                           o_span[1] + 1 if o_span[1] != -1 else -1)

                labels_array = [invalid_class] * len(category_list)
                is_valid = False
                
                for q in quads:
                    if tuple(q.get('span_A', (-1, -1))) == a_span and tuple(q.get('span_B', (-1, -1))) == o_span:
                        cat_id = category2id[q['category_aspect']] 
                        sent_id = int(q['sentiment'])              
                        labels_array[cat_id] = sent_id
                        is_valid = True
                
                # Evitiamo di creare (NULL, NULL) finti per non inquinare troppo i dati
                if not is_valid and a_span == (-1, -1) and o_span == (-1, -1):
                    continue
                
                new_data.append({
                    'review_text': text,
                    'aspect_span': fixed_a,     
                    'opinion_span': fixed_o,    
                    'labels': labels_array       
                })
                
    return pd.DataFrame(new_data)

# --- 3. FUNZIONE MASTER ---
def process_and_save_domain(domain_name, df_train, df_dev, df_test, output_dir="data_coppie"):
    """
    Esegue l'intera pipeline per un dominio specifico e salva i file su disco.
    """
    print(f"\nAvvio pipeline per il dominio: {domain_name.upper()}...")
    
    # 1. Troviamo le categorie
    category_list, category2id = build_category_vocab([df_train, df_dev, df_test])
    print(f"Trovate {len(category_list)} categorie uniche.")
    
    # 2. Creiamo i dataframe delle coppie
    print(f"Generazione dei dataset di Coppie in corso...")
    df_train_pairs = prepare_pairs_dataset(df_train, category_list, category2id)
    df_dev_pairs = prepare_pairs_dataset(df_dev, category_list, category2id)
    df_test_pairs = prepare_pairs_dataset(df_test, category_list, category2id)
    
    # 3. Salvataggio
    os.makedirs(output_dir, exist_ok=True)
    
    df_train_pairs.to_pickle(os.path.join(output_dir, f"train_{domain_name}_pairs.pkl"))
    df_dev_pairs.to_pickle(os.path.join(output_dir, f"dev_{domain_name}_pairs.pkl"))
    df_test_pairs.to_pickle(os.path.join(output_dir, f"test_{domain_name}_pairs.pkl"))
    
    with open(os.path.join(output_dir, f"{domain_name}_categories.pkl"), "wb") as f:
        pickle.dump(category_list, f)
        
    print(f"Dati per '{domain_name}' salvati in '{output_dir}'! Esempi nel Train: {len(df_train_pairs)}")
    return df_train_pairs, df_dev_pairs, df_test_pairs


In [7]:
# Lanciamo per il Laptop
train_lap, dev_lap, test_lap = process_and_save_domain("laptop", df_train_parsing_laptop, df_dev_parsing_laptop, df_test_parsing_laptop)
display(train_lap.head(2))

# Lanciamo per il Restaurant
train_rest, dev_rest, test_rest = process_and_save_domain("restaurant", df_train_parsing_rest, df_dev_parsing_rest, df_test_parsing_rest)
display(train_rest.head(2))


Avvio pipeline per il dominio: LAPTOP...
Trovate 121 categorie uniche.
Generazione dei dataset di Coppie in corso...
Dati per 'laptop' salvati in 'data_coppie'! Esempi nel Train: 9247


,review_text,aspect_span,opinion_span,labels
0,ace ##r wants $ 170 to just look at it then ad...,"(1, 3)","(-1, -1)","[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, ..."
1,update : i repaired it myself for $ 12 .,"(-1, -1)","(-1, -1)","[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, ..."



Avvio pipeline per il dominio: RESTAURANT...
Trovate 13 categorie uniche.
Generazione dei dataset di Coppie in corso...
Dati per 'restaurant' salvati in 'data_coppie'! Esempi nel Train: 6366


,review_text,aspect_span,opinion_span,labels
0,judging from previous posts this used to be a ...,"(11, 12)","(14, 17)","[3, 3, 3, 3, 3, 3, 3, 3, 3, 0, 3, 3, 3]"
1,judging from previous posts this used to be a ...,"(11, 12)","(-1, -1)","[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]"
